In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json  as js
from geopandas import GeoDataFrame
from shapely.geometry import Point
import datetime as dt
import glob
import csv
from dateutil import parser
import os

In [3]:
b = pd.read_csv('report_f_all.csv', sep = ';', encoding='utf8')
a = gpd.read_file('557_zone.SHP')

In [4]:
a = gpd.read_file('557_zone.SHP')
a['zid_from']=a['NO']
a['zid_to']=a['NO']

In [5]:
b2=b[b['buffer_out_ind']==1]
b2=b2[b2['30_min_ind']==1]
b2=b2[['time_key','ts','zid_from','zid_to','count_ctn']]

In [6]:
prop = pd.read_csv('prop2.csv',sep = ';', encoding='utf-8')
prop['zid_from']=prop['Номер транспортного района']
prop['zid_to']=prop['Номер транспортного района']
prop=prop.drop(['Номер транспортного района'],axis=1)
prop=prop.drop(['Муниципалитет (нижний уровень)'],axis=1)
prop=prop.drop(['Город/ Муниципалитет'],axis=1)

In [7]:
b3=b2[b2['time_key']=='2019-04-11']

In [32]:
l=b3.merge(prop, on='zid_from')

In [33]:
l=l.drop(['zid_to_y'],axis=1)
l['FROM']=l['Пригород чего']
l['Км от 156 района_FROM']=l['Км от 156 района']
l=l.drop(['Пригород чего'],axis=1).drop(['Км от 156 района'],axis=1)
l['zid_to']=l['zid_to_x']
l=l.drop(['zid_to_x'],axis=1)

In [57]:
l2=l.merge(prop, on='zid_to')
l2['TO']=l2['Пригород чего']
l2=l2.drop(['Км от 156 района'],axis=1)
l2=l2.drop(['Пригород чего'],axis=1)
l2['zid_from']=l2['zid_from_x']

In [58]:
Samara_Tol=l2[(l2['TO']=='городской округ Самара') | (l2['TO']=='городской округ Тольятти')]

In [59]:
Samara_Tol['zid_from'].unique()

array([  1,   2,   4,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,
        16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  53,  54,  56,  57,
        58,  59,  61,  62,  65,  66,  67,  68,  69,  71,  72,  73,  74,
        75,  76,  78,  81,  82,  85,  86,  87,  88,  89,  90,  91,  95,
        96,  97, 100, 101, 106, 107, 115, 116, 119, 120, 121, 122, 123,
       125, 126, 127, 128, 130, 131, 132, 134, 135, 138, 140, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 170,
       171, 172, 173, 174, 178, 179, 181, 182, 183, 184, 185, 187, 189,
       190, 191, 193, 196, 199, 210, 212, 216, 217, 220, 223, 224, 225,
       226, 227, 228, 229, 230, 231, 233, 238, 242, 246, 248, 249, 251,
       257, 258, 259, 260, 261, 264, 265, 266, 269, 270, 272, 27

In [60]:
Samara_Tol=pd.DataFrame(Samara_Tol.groupby(['zid_from','FROM','TO']).agg({'count_ctn': np.sum})).reset_index()

In [61]:
Samara_Tol.to_csv('res.csv')

In [53]:
Samara_Tol.сolumns=['zid_from','zid_to','FROM','TO','count_ctn']

C:\Users\makol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [54]:
Samara_Tol2=Samara_Tol.merge(a, on='zid_from').drop(['NO'], axis=1)


In [55]:
total1 = gpd.GeoDataFrame(Samara_Tol2, geometry='geometry')
total1.crs='epsg:4326'
total1.to_file('Samara_to_Tolyat.shp', driver='ESRI Shapefile',encoding="utf-8")